<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/pytorch/60_minute_blitz/torch_autograd_Introduction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`torch.autograd`的简要介绍
=========================================

`torch.autograd` 是 PyTorch 的自动微分引擎，为神经网络训练提供动力。在本节中，您将对 autograd 如何帮助神经网络训练有一个概念性的了解。

背景
----------

神经网络 (NN) 是对某些输入数据执行的嵌套函数的集合。这些函数由参数（由权重和偏差组成）定义，这些参数在 PyTorch 中存储在张量中。

训练神经网络分两步进行：:

**Forward Propagation（前向传播）**: 在前向传播中，神经网络对正确的输出做出最佳猜测。它通过每个函数运行输入数据来进行猜测。

**Backward Propagation（反向传播）**: 在反向传播中，神经网络根据其猜测的误差按比例调整其参数。它通过从输出向后遍历，收集误差相对于函数参数（梯度）的导数，并使用梯度下降优化参数来实现这一点。有关反向传播的更详细演练，请观看 [
3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

在 PyTorch 中的用法
----------------

让我们看一下单个训练步骤。对于此示例，我们从 torchvision 加载预训练的 resnet18 模型。我们创建一个随机数据张量来表示具有 3 个通道、高度和宽度为 64 的单个图像，并将其相应的 label 初始化为一些随机值。预训练模型中的标签具有形状 (1,1000)。.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>本教程仅适用于 CPU，不适用于 GPU 设备（即使张量移至 CUDA）。.</p>
</div>


In [1]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 45.3MB/s]


接下来，我们通过模型的每一层运行输入数据以进行预测。这就是**向前传递**。


In [2]:
prediction = model(data) # forward pass

我们使用模型的预测和相应的标签来计算误差（ loss ）。下一步是通过网络反向传播此错误。当我们对误差张量调用 .backward() 时，反向传播就开始了。然后，Autograd 计算每个模型参数的梯度并将其存储在参数的 .grad 属性中。


In [3]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

接下来，我们加载优化器，在本例中为 SGD，学习率为 0.01，动量为 0.9。我们在优化器中注册模型的所有参数



In [4]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

最后，我们调用 .step() 来启动梯度下降。优化器通过存储在 .grad 中的梯度来调整每个参数。


In [5]:
optim.step() #gradient descent

此时，您已拥有训练神经网络所需的一切。以下部分详细介绍了 autograd 的工作原理 - 可以跳过它们。


------------------------------------------------------------------------


Autograd 中的差异化
===========================

我们来看看 autograd 是如何收集梯度的。我们使用 requires_grad=True 创建两个张量 a 和 b 。这向 autograd 发出信号，应跟踪对它们的每个操作。


In [6]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

我们从 a 和 b 创建另一个张量 Q 。

$$Q = 3a^3 - b^2$$


In [7]:
Q = 3*a**3 - b**2

我们假设 a 和 b 是神经网络的参数， Q 是错误。在神经网络训练中，我们需要误差的梯度。参数，即



$$\frac{\partial Q}{\partial a} = 9a^2$$

$$\frac{\partial Q}{\partial b} = -2b$$

当我们在 Q 上调用 .backward() 时，autograd 会计算这些梯度并将它们存储在相应张量的 .grad 属性中。

我们需要在 Q.backward() 中显式传递 gradient 参数，因为它是一个向量。 gradient 是与 Q 形状相同的张量，它表示 Q w.r.t 的梯度。本身，即

$$\frac{dQ}{dQ} = 1$$

同样，我们也可以将 Q 聚合为标量并隐式向后调用，如 Q.sum().backward() 。


In [9]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

梯度现在存放在 a.grad 和 b.grad 中


In [10]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


计算图
===================

从概念上讲，autograd 在由 Function 对象组成的有向无环图 (DAG) 中保存数据（张量）和所有执行的操作（生成的新张量）的记录。在这个 DAG 中，叶子是输入张量，根是输出张量。通过从根到叶追踪该图，您可以使用链式法则自动计算梯度。

在前向传递中，autograd 同时执行两件事：:

-   运行请求的操作来计算结果张量
-   在 DAG 中维护操作的梯度函数。

当在 DAG 根上调用 .backward() 时，向后传递开始。 autograd 然后:

-   计算每个 .grad_fn 的梯度，
-   将它们累积到各自张量的 .grad 属性中
-   使用链式法则，一直传播到叶张量。

下面是我们示例中 DAG 的直观表示。图中，箭头指向前向传递的方向。节点代表前向传递中每个操作的后向函数。蓝色的叶节点代表我们的叶张量 a 和 b 。

![](https://pytorch.org/tutorials/_static/img/dag_autograd.png)

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>需要注意的重要一点是图是从头开始重新创建的；每次 .backward() 调用后，autograd 开始填充新图表。这正是允许您在模型中使用控制流语句的原因；如果需要，您可以在每次迭代时更改形状、大小和操作。</p>
</div>

从 DAG 中排除
----------------------

torch.autograd 跟踪所有 requires_grad 标志设置为 True 的张量上的操作。对于不需要梯度的张量，将此属性设置为 False 会将其从梯度计算 DAG 中排除。

即使只有一个输入张量具有 requires_grad=True ，操作的输出张量也将需要梯度。


In [11]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients?: False
Does `b` require gradients?: True


在神经网络中，不计算梯度的参数通常称为**冻结参数**。如果您事先知道不需要这些参数的梯度，那么“冻结”模型的一部分会很有用（这通过减少自动梯度计算来提供一些性能优势）。

在微调中，我们冻结大部分模型，通常只修改分类器层以对新标签进行预测。让我们通过一个小例子来演示这一点。和之前一样，我们加载预训练的 resnet18 模型，并冻结所有参数。


In [12]:
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

假设我们想要在具有 10 个标签的新数据集上微调模型。在 resnet 中，分类器是最后一个线性层 model.fc 。我们可以简单地用一个新的线性层（默认情况下未冻结）替换它作为我们的分类器。


In [13]:
model.fc = nn.Linear(512, 10)

现在模型中的所有参数（除了 model.fc 的参数）都被冻结。计算梯度的唯一参数是 model.fc 的权重和偏差。


In [14]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

请注意，虽然我们在优化器中注册了所有参数，但计算梯度（因此在梯度下降中更新）的唯一参数是分类器的权重和偏差。



Further readings:
=================

-   [In-place operations & Multithreaded
    Autograd](https://pytorch.org/docs/stable/notes/autograd.html)
-   [Example implementation of reverse-mode
    autodiff](https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC)
-   [Video: PyTorch Autograd Explained - In-depth
    Tutorial](https://www.youtube.com/watch?v=MswxJw-8PvE)
